In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install timm

In [4]:
%cd '/content/drive/MyDrive/인지프'
%pwd

/content/drive/MyDrive/인지프


'/content/drive/MyDrive/인지프'

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import time
import tqdm
from tqdm.notebook import tqdm as notebooktqdm
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from PIL import Image
import timm
from timm.layers import BatchNormAct2d
import os
from encoding_module import EcodingModel

In [14]:
# work place
work_dir = './'
os.chdir(work_dir)

In [7]:
class YoutubeDataset(Dataset):
    def __init__(self, data, doc2vec):
        self.ids = list(data['video_id'])
        self.titles = doc2vec # pretrained doc2vec features
        self.data = data # video_id, metadata, views(y) from csv file
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.image_encoder = timm.create_model('efficientnet_b1_pruned', features_only =True, pretrained=True)
        model = timm.create_model('efficientnet_b1_pruned', pretrained=True)
        data_cfg = timm.data.resolve_data_config(model.pretrained_cfg)
        self.transform = timm.data.create_transform(**data_cfg)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        # index order of video_id, meta, y are same
        video_id = self.ids[idx]
        
        image = Image.open( work_dir+'medium_15287/{}.jpg'.format(video_id))
        image = self.transform(image)
        # image = torch.FloatTensor(np.array(image)).permute(2, 0, 1).unsqueeze(dim=0)
        self.image_encoder.eval()
        feature_map = self.image_encoder(torch.unsqueeze(image,0))[-1].squeeze() # (320,6,10)
        
        title = self.titles[video_id] # get video title
        title = torch.FloatTensor(np.array(title, dtype=np.float16))
        
        meta = torch.FloatTensor(self.data[['period_day', 'subscriber_count']].to_numpy()[idx]) # get metadata
        
        y = np.log10(self.data['views'].to_numpy() + 1) # add 1 for zero views
        y = np.expand_dims(y, axis=1) # add batch dimension
        y = torch.FloatTensor(y[idx]) # get log10(views+1) by idx value
        
        return video_id, feature_map, title, meta, y

In [8]:
# add nomarlizing
data = pd.read_csv('./train.csv')
mean_period = data['period_day'].mean()
std_period = data['period_day'].std()
mean_sub = data['subscriber_count'].mean()
std_sub = data['subscriber_count'].std()
print(mean_period, std_period, mean_sub, std_sub)

data['period_day'] = (data['period_day'] - mean_period)/std_period
data['subscriber_count'] = (data['subscriber_count']-mean_sub)/std_sub

train_data, valid_data = train_test_split(data, test_size = 0.1, random_state = 55)
test_data = pd.read_csv('./test.csv')
# train_data = train_data[:1000]
# valid_data = valid_data[:100]
# test_data = test_data[:100]
print('Train Dataset Size : ',len(train_data))
print('Validation Dataset Size : ',len(valid_data))
print('Test Dataset Size : ',len(test_data))

# data.head()

335.8148713475796 497.7613157973895 1784323.5617822357 3833786.6144638904
Train Dataset Size :  12382
Validation Dataset Size :  1376
Test Dataset Size :  1529


In [9]:
# open doc2vec data and conver to dict
with open('./title_doc2vec_10', 'rb') as f:
    doc2vec = pickle.load(f)

data_dict=dict()
for row in doc2vec:
    vid=row[0]
    vec=row[1:]
    data_dict[vid]=vec

doc2vec = data_dict
print(len(doc2vec))

15287


In [10]:
#setting hyper parameters
batch_size = 64
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
train_dataset = YoutubeDataset(train_data, doc2vec)
valid_dataset = YoutubeDataset(valid_data, doc2vec)
test_dataset = YoutubeDataset(test_data, doc2vec)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size = 1)
test_loader = DataLoader(test_dataset, batch_size = 1)

In [16]:
# !unzip -o ./medium_15287.zip -d ./medium_15287

data_list = os.listdir('./medium_15287')
print(len(data_list))

9495


In [13]:
encoder = EcodingModel()
encoder.to(encoder.device)
train_img_title, train_title, train_meta, train_target = encoder.encode(train_loader)

FileNotFoundError: ignored

In [ ]:
data = np.concatenate(train_img_title, train_title, train_meta)
train_df = pd.DataFrame(train_img_title, columns=['img_title_{}'.format(i) for i in range(10)] + ['title_{}'.format(i) for i in range(10)] + ['period_day', 'subscriber_count'])

In [ ]:
from xgboost import XGBClassifier
from xgboost import plot_importance

xgbc = XGBClassifier(random_state=111)
xgbc.fit(train_df, train_target)

plot_importance(xgbc, max_num_features=10)